In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import ElasticNet, ElasticNetCV, LassoCV, Lasso
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler, RobustScaler, scale
import seaborn as sns 

In [2]:
tracts_deserts_path = 'data/tract_desert_measures/atlanta_desert_tracts.csv'
place_abbr = 'atlanta'
save_data_name = 'data/tract_health' + place_abbr+'_tract_health.csv'
save_transf_data_name = 'data/tract_health' + place_abbr+'_tract_health_logt.csv'
save_results_name = 'data/tract_health/results' + place_abbr+'_tract_lasso_results.csv'

# Aggregation/Cleaning

In [3]:
tracts_deserts = pd.read_csv(tracts_deserts_path, dtype={'GEOID':'str'})
tracts_deserts = tracts_deserts.rename(columns={'GEOID':'TractFIPS'})

In [4]:
tracts_health = pd.read_csv("data/PLACES__Census_Tract_Data__GIS_Friendly_Format___2021_release.csv", dtype={'TractFIPS':'str'})
tracts_health = pd.merge(tracts_health, tracts_deserts, on='TractFIPS')
tracts_health.to_csv(save_data_name, index=False)

In [5]:
desert_measures = ['food_closest_travel_times', 'physical_closest_dist', 'transport_closest_dist', 'education_closest_travel_times', 'worship_closest_travel_times']

tracts_health_logt = tracts_health.copy()
tracts_health_logt[desert_measures] = tracts_health_logt[desert_measures].apply(lambda x: np.log(x+1))
tracts_health_logt.to_csv(save_transf_data_name, index=False)

# Analysis

In [6]:
name_mapping = {'access2': 'Health insurance access', 'arthritis': 'Arthritis prevalence', 'binge': 'Binge drinking prevalence',
               'bphigh': 'High blood pressure prevalence', 'bpmed': 'Medium blood pressure prevalence', 'cancer': 'Cancer prevalence',
               'casthma': 'Asthma prevalence', 'cervical': 'Cervical cancer screenings', 'chd': 'Coronary heart disease prevalence',
               'checkup': 'Routine checkups', 'cholscreen': 'Cholesterol screenings', 'colon_screen': 'Colon cancer screenings',
               'copd': 'COPD prevalence', 'corem': 'Core men\'s health', 'corew': 'Core women\'s health', 'csmoking': 'Smoking prevalence',
               'dental': 'Dental checkups', 'depression': 'Depression prevalence', 'diabetes': 'Diabetes prevalence', 'ghlth': 'General poor health prevalence',
               'highchol': 'High cholesterol prevalence', 'kidney': 'Chronic kidney disease', 'lpa': 'No physical activity', 'mammouse': 'Mammograms',
               'mhlth': 'Poor mental health prevalence', 'obesity': 'Obesity prevalence', 'phlth': 'Poor physical health', 'sleep': 'Poor sleep prevalence',
               'stroke': 'Stroke prevalence', 'teethlost': 'Teeth loss prevalence'}

results_nan = pd.DataFrame({'Health condition': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')], 'Food': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')],
                       'Physical health': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')], 'Public transport': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')],
                        'Education': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')], 'Houses of worship': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')],
                       'RSquared': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')], 'MSE': [np.nan for c in tracts_health.columns if c.endswith('CrudePrev')]})


In [7]:
tracts_health

StateAbbr StateDesc CountyName  CountyFIPS    TractFIPS  TotalPopulation  \
0          GA   Georgia     Fulton       13121  13121010204             4761   
1          GA   Georgia     Fulton       13121  13121005501             2307   
2          GA   Georgia     Fulton       13121  13121001202             3937   
3          GA   Georgia     Fulton       13121  13121011425             6906   
4          GA   Georgia     Fulton       13121  13121009403             4625   
..        ...       ...        ...         ...          ...              ...   
197        GA   Georgia     Fulton       13121  13121011623            11473   
198        GA   Georgia     Fulton       13121  13121000600             5203   
199        GA   Georgia     Fulton       13121  13121009603             4388   
200        GA   Georgia     Fulton       13121  13121003900             1331   
201        GA   Georgia     Fulton       13121  13121001201             3479   

     ACCESS2_CrudePrev ACCESS2_Crude95CI  ARTHRITIS_CrudePrev  \
0                  9.6      ( 7.5, 12.7)                 24.8   
1                 25.2      (21.3, 29.6)                 20.0   
2                 12.6      (11.5, 14.2)                 10.0   
3                 10.8      ( 8.9, 13.5)                 20.0   
4                 18.4      (15.7, 21.9)                 13.5   
..                 ...               ...                  ...   
197               13.6      (11.9, 15.4)                 16.1   
198               20.0      (17.9, 22.5)                  8.4   
199               14.2      (12.2, 17.5)                 15.5   
200               29.2      (25.2, 33.5)                 26.8   
201               13.8      (11.8, 16.5)                 12.7   

    ARTHRITIS_Crude95CI  ...  STROKE_CrudePrev STROKE_Crude95CI  \
0          (23.4, 26.2)  ...               2.6     ( 2.3,  3.1)   
1          (18.9, 20.9)  ...               3.9     ( 3.4,  4.4)   
2          ( 9.6, 10.4)  ...               1.1     ( 1.0,  1.2)   
3          (18.9, 21.3)  ...               2.1     ( 1.8,  2.4)   
4          (12.9, 14.1)  ...               1.9     ( 1.7,  2.1)   
..                  ...  ...               ...              ...   
197        (15.5, 16.9)  ...               1.8     ( 1.7,  2.0)   
198        ( 8.1,  8.7)  ...               1.2     ( 1.1,  1.3)   
199        (14.9, 16.2)  ...               1.8     ( 1.6,  2.0)   
200        (25.9, 27.8)  ...               6.4     ( 5.9,  7.1)   
201        (12.3, 13.2)  ...               1.5     ( 1.4,  1.7)   

     TEETHLOST_CrudePrev TEETHLOST_Crude95CI  \
0                    4.4        ( 2.6,  8.2)   
1                   23.5        (14.5, 33.7)   
2                    4.6        ( 3.4,  6.4)   
3                    4.8        ( 2.8,  7.7)   
4                    9.8        ( 5.9, 17.5)   
..                   ...                 ...   
197                  5.8        ( 3.9,  8.6)   
198                 13.4        (10.3, 17.5)   
199                  5.9        ( 3.5, 11.6)   
200                 29.4        (20.8, 39.1)   
201                  7.2        ( 4.5, 11.3)   

                          Geolocation food_closest_travel_times  \
0    POINT (-84.41716366 33.92519129)                     258.6   
1    POINT (-84.38372162 33.72987572)                     130.1   
2    POINT (-84.38515368 33.77682317)                      72.2   
3    POINT (-84.24164477 34.01149372)                     289.4   
4    POINT (-84.35991176 33.83156797)                     169.2   
..                                ...                       ...   
197  POINT (-84.19032903 34.03959659)                     289.4   
198  POINT (-84.40584642 33.78496362)                     119.0   
199  POINT (-84.37581823 33.83437332)                      50.4   
200  POINT (-84.42210552 33.75005537)                     115.2   
201  POINT (-84.38029279 33.77637729)                      74.7   

     physical_closest_dist transport_closest_dist  \
0                 1.05

In [8]:
alphas = []
l1_ratios = []
i=0
results = results_nan.copy()

plt.figure(figsize=(20, 15))
plt.subplots_adjust(hspace=0.5)

for c in tracts_health_logt.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        
        x = tracts_health_logt[desert_measures].to_numpy(copy=True)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = tracts_health_logt[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        data_centered = center_function(y)
        
        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.25, 
                                                    random_state=42)
        
        #l1 ratio is from suggested values in ElasticNetCV documentation
        enet_cv = ElasticNetCV(l1_ratio = [.1, .5, .7, .9, .95, .99, 1], 
                                     cv = 10, normalize=True).fit(X_train,y_train)
        
        alpha = enet_cv.alpha_
        l1 = enet_cv.l1_ratio_
        alphas.append(alpha)
        l1_ratios.append(l1)
        
        regr = ElasticNet(alpha=alpha, l1_ratio = l1, normalize=True)  # Could try others, or other parameters?
        regr.fit(X_train, y_train.reshape(-1, 1))
        
        predictions = regr.predict(X_test)
        y_train_pred = regr.predict(X_train)
        mse_test = mean_squared_error(y_test, predictions)
        
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[1]
        results.iat[i, 2] = regr.coef_[3]
        results.iat[i, 3] = regr.coef_[4]
        results.iat[i, 4] = regr.coef_[0]
        results.iat[i, 5] = regr.coef_[2]
        results.iat[i, 6] = regr.score(X_test, y_test)
        results.iat[i, 7] = mse_test
            
        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

Health condition    Food  Physical health  \
0                          Mammograms -0.2212          -0.6048   
1                    Routine checkups -0.0000          -1.6251   
2                  Obesity prevalence -0.1687          -2.9856   
3                   Core men's health  0.3283           2.5598   
4               Poor sleep prevalence -0.4577          -2.1172   
5      High blood pressure prevalence  0.0000          -4.6510   
6              Cholesterol screenings  0.2699          -0.0000   
7          Cervical cancer screenings  0.2752           0.3984   
8               Teeth loss prevalence -0.3015          -2.6953   
9                     Dental checkups  0.3294           3.5585   
10                  Asthma prevalence -0.1911          -0.7213   
11     General poor health prevalence  0.0000          -2.9676   
12                Diabetes prevalence  0.0000          -2.5698   
13               Poor physical health  0.1119          -1.4323   
14          Binge drinking prevalence -0.0000           1.3959   
15               No physical activity -0.0000          -2.9324   
16        High cholesterol prevalence  0.7146          -0.6079   
17                    COPD prevalence  0.1119          -0.7636   
18      Poor mental health prevalence -0.3171          -0.8722   
19             Chronic kidney disease  0.0000          -0.6095   
20                  Stroke prevalence  0.0000          -0.9153   
21                 Smoking prevalence -0.1505          -1.3965   
22               Arthritis prevalence  0.5477          -1.9023   
23            Health insurance access  0.0000          -1.5893   
24            Colon cancer screenings  0.0000           0.7681   
25  Coronary heart disease prevalence  0.0950          -0.7322   
26                Core women's health  0.2559           3.0829   
27                  Cancer prevalence  0.2033          -0.0725   
28              Depression prevalence -0.1362           0.0494   
29   Medium blood pressure prevalence  0.9269          -2.5511   

    Public transport  Education  Houses of worship  RSquared         MSE  
0            -0.1667    -0.3088           0.338724    0.3640    1.330052  
1             0.0740     0.6232           2.368616    0.1867   10.062470  
2            -1.2830     1.1287           2.646939    0.1866   50.395890  
3             1.4580    -0.6289          -1.507327    0.1783   45.257121  
4            -1.4277     0.5053           2.083155    0.1567   36.461682  
5            -0.0447     2.1743           4.183252    0.1551   73.709136  
6             0.5549     0.1215           1.035238    0.1497    9.495159  
7             0.4731    -0.1084           0.474314    0.1470    8.386475  
8            -1.6579     1.5909           1.010453    0.1453   75.071514  
9             2.6162    -1.5262          -1.847335    0.1423  162.545397  
10           -0.3533     0.2491           0.609936    0.1384    2.911510  
11           -1.2908     1.7799           1.851936    0.1332   72.855358  
12           -0.1010     1.2936           1.880785    0.1311   24.260682  
13           -0.5847     1.0264           0.874152    0.1261   18.394164  
14            0.0000    -0.8751          -1.723921    0.1257    8.789642  
15           -1.4627     1.6540           1.942381    0.1225   76.327932  
16            0.5617     1.2055           1.152699    0.1113   17.605054  
17           -0.2152     0.6231           0.446602    0.1103    5.401611  
18           -0.8674     0.3490           0.219566    0.1101   15.866169  
19           -0.0000     0.3271           0.370488    0.1062    1.423246  
20           -0.0000     0.4728           0.553436    0.1057    3.107651  
21           -1.1256     0.8788           0.892329    0.1033   33.236421  
22            0.2908     1.5628           1.893694    0.0995   24.603220  
23           -1.5186     0.9025           0.865958    0.0937   65.438797  
24            1.3601    -0.5162           0.000000    0.0934   43.371715  
25            0.0000    

<Figure size 1440x1080 with 0 Axes>

In [9]:
alphas = []
l1_ratios = []
i=0
results = results_nan.copy()

plt.figure(figsize=(20, 15))
plt.subplots_adjust(hspace=0.5)

for c in tracts_health.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        
        x = tracts_health[desert_measures].to_numpy(copy=True)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = tracts_health[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        y = center_function(y)
        
        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)
        
        #l1 ratio is from suggested values in ElasticNetCV documentation
        lasso_cv = LassoCV(cv = 5, normalize=True).fit(X_train,y_train)
        
        alpha = lasso_cv.alpha_
        alphas.append(alpha)
        
        regr = Lasso(alpha=alpha, normalize=True)  # Could try others, or other parameters?
        regr.fit(X_train, y_train.reshape(-1, 1))
        
        predictions = regr.predict(X_test)
        y_train_pred = regr.predict(X_train)
        mse_test = mean_squared_error(y_test, predictions)
        
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[1]
        results.iat[i, 2] = regr.coef_[3]
        results.iat[i, 3] = regr.coef_[4]
        results.iat[i, 4] = regr.coef_[0]
        results.iat[i, 5] = regr.coef_[2]
        results.iat[i, 6] = regr.score(X_test, y_test)
        results.iat[i, 7] = mse_test

        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

Health condition    Food  Physical health  \
0                          Mammograms -0.3298          -1.0164   
1                 Core women's health  0.0000           3.4246   
2                    Routine checkups  0.1485          -2.3979   
3                  Obesity prevalence  0.0000          -5.3847   
4      High blood pressure prevalence  0.7142          -6.7168   
5                   Core men's health  0.1390           4.7395   
6               Poor sleep prevalence -0.5867          -4.0842   
7                 Diabetes prevalence  0.3631          -3.7039   
8                   Stroke prevalence  0.1357          -1.3538   
9                   Asthma prevalence -0.1491          -1.4195   
10             Chronic kidney disease  0.1108          -0.9666   
11          Binge drinking prevalence -0.3187           2.1052   
12             Cholesterol screenings  0.3872           0.0000   
13                    Dental checkups  0.0000           8.0837   
14               Arthritis prevalence  1.0292          -3.4244   
15        High cholesterol prevalence  1.2274          -0.8302   
16     General poor health prevalence  0.2255          -6.0356   
17               No physical activity  0.0000          -5.8067   
18                 Smoking prevalence  0.0000          -3.2075   
19              Teeth loss prevalence  0.0000          -6.2428   
20                  Cancer prevalence  0.2484          -0.0000   
21               Poor physical health  0.2788          -3.1041   
22   Medium blood pressure prevalence  1.6315          -3.3186   
23                    COPD prevalence  0.2469          -1.5377   
24  Coronary heart disease prevalence  0.3622          -1.4883   
25            Health insurance access  0.0000          -3.6675   
26              Depression prevalence -0.0000          -0.0000   
27      Poor mental health prevalence -0.3655          -2.4186   
28            Colon cancer screenings -0.0000           1.9262   
29         Cervical cancer screenings  0.2499           1.2841   

    Public transport  Education  Houses of worship  RSquared         MSE  
0            -0.0000     0.0691           0.705301    0.4960    1.105436  
1             0.5330    -1.4371          -1.756193    0.2726   28.951943  
2             0.5169     1.5150           1.916573    0.2528    9.792345  
3            -0.3848     3.3402           2.961345    0.2339   46.685688  
4             0.0207     4.6741           3.459120    0.2299   68.003822  
5             0.8402    -2.6138          -2.172101    0.1927   42.586556  
6            -1.0984     2.5067           3.195386    0.1815   33.894499  
7             0.0000     2.7296           1.397040    0.1800   23.025880  
8             0.0000     0.9864           0.435177    0.1630    2.919132  
9            -0.1684     0.8234           0.825877    0.1552    2.859230  
10            0.0497     0.6886           0.281465    0.1455    1.387426  
11           -0.1075    -1.7585          -1.278814    0.1432    9.532976  
12            0.6614     0.1493           0.630375    0.1365    9.190320  
13            1.2612    -5.1483          -2.986282    0.1109  159.280828  
14            0.8619     2.6584           1.178983    0.1080   24.793925  
15            0.6334     1.4183           0.078136    0.1075   18.123702  
16           -0.2299     4.3144           2.026075    0.1010   74.481839  
17           -0.3356     4.1469           2.208286    0.0937   77.598792  
18           -0.7695     2.3339           1.434410    0.0897   32.279675  
19           -0.4857     4.2138           1.409819    0.0804   76.312432  
20            0.3257     0.1098           0.000000    0.0793    2.618643  
21           -0.0038     2.3029           0.894442    0.0782   19.286322  
22            1.2101     3.0094           1.833742    0.0782   41.091465  
23           -0.0000     1.2383           0.334406    0.0780    5.591660  
24            0.2978     1.0758           0.204237    0.0686    3.817102  
25           -1.0077    

<Figure size 1440x1080 with 0 Axes>

In [10]:
alphas = []
l1_ratios = []
i=0
results = results_nan.copy()

plt.figure(figsize=(20, 15))
plt.subplots_adjust(hspace=0.5)

for c in tracts_health_logt.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        
        x = np.log(tracts_health[desert_measures].to_numpy(copy=True)+1)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = tracts_health_logt[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        y = center_function(y)
        
        X_train, X_test, y_train, y_test = train_test_split(xscale, 
                                                    y, 
                                                    test_size=0.3, 
                                                    random_state=42)
        
        #l1 ratio is from suggested values in ElasticNetCV documentation
        lasso_cv = LassoCV(cv = 5, normalize=True).fit(X_train,y_train)
        
        alpha = lasso_cv.alpha_
        alphas.append(alpha)
        
        regr = Lasso(alpha=alpha, normalize=True)  # Could try others, or other parameters?
        regr.fit(X_train, y_train.reshape(-1, 1))
        
        predictions = regr.predict(X_test)
        y_train_pred = regr.predict(X_train)
        mse_test = mean_squared_error(y_test, predictions)
        
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[1]
        results.iat[i, 2] = regr.coef_[3]
        results.iat[i, 3] = regr.coef_[4]
        results.iat[i, 4] = regr.coef_[0]
        results.iat[i, 5] = regr.coef_[2]
        results.iat[i, 6] = regr.score(X_test, y_test)
        results.iat[i, 7] = mse_test

        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4})
results_round.sort_values(by='RSquared', ascending=False).reset_index(drop=True)

Health condition    Food  Physical health  \
0                          Mammograms -0.2812          -0.6970   
1              Cholesterol screenings  0.1393          -0.0000   
2                   Core men's health  0.0000           3.2088   
3          Cervical cancer screenings  0.0000           0.5272   
4                    Routine checkups  0.0000          -2.2174   
5       Poor mental health prevalence -0.0000          -0.9555   
6                  Obesity prevalence -0.0000          -4.1302   
7         High cholesterol prevalence  0.8529          -1.5876   
8                     Dental checkups  0.0000           5.3600   
9      High blood pressure prevalence  0.0000          -5.2841   
10                  Cancer prevalence  0.2732          -0.3524   
11            Colon cancer screenings  0.0000           1.0884   
12          Binge drinking prevalence -0.0000           1.6566   
13              Poor sleep prevalence -0.6638          -3.1864   
14              Depression prevalence -0.0959           0.0000   
15            Health insurance access  0.0000          -2.3294   
16   Medium blood pressure prevalence  0.8845          -3.5755   
17                Diabetes prevalence  0.0000          -2.8760   
18                 Smoking prevalence -0.0000          -2.1998   
19              Teeth loss prevalence -0.0000          -4.2974   
20                Core women's health  0.1261           3.5279   
21               No physical activity  0.0000          -4.4682   
22     General poor health prevalence  0.0000          -4.6456   
23                  Stroke prevalence  0.0039          -1.0674   
24             Chronic kidney disease  0.0106          -0.7322   
25  Coronary heart disease prevalence  0.1934          -1.1053   
26               Poor physical health  0.0000          -2.2893   
27               Arthritis prevalence  0.6001          -3.1981   
28                  Asthma prevalence -0.2239          -1.1166   
29                    COPD prevalence  0.0570          -1.2663   

    Public transport  Education  Houses of worship  RSquared         MSE  
0            -0.1770    -0.3498           0.463365    0.3413    1.444706  
1             0.5067     0.0007           1.150573    0.1842    8.682611  
2             1.2172    -0.5095          -1.564761    0.1465   45.022378  
3             0.4017    -0.0000           0.600197    0.1262    8.051690  
4             0.2235     0.7024           2.627308    0.1156   11.591033  
5            -0.7273     0.0602           0.000000    0.1137   15.368493  
6            -1.0365     1.1308           3.257497    0.1002   54.832320  
7             0.8235     1.5285           1.670560    0.0976   18.325025  
8             2.2012    -1.6189          -2.573989    0.0776  165.236244  
9            -0.0000     2.0366           4.358984    0.0765   81.549844  
10            0.4714     0.3921           0.351310    0.0715    2.640722  
11            1.2537    -0.5629          -0.000000    0.0669   41.340661  
12            0.0000    -0.8582          -1.809819    0.0653   10.398858  
13           -1.3822     0.6387           3.087723    0.0604   38.907459  
14           -0.0757     0.0932          -0.185795    0.0532    1.480275  
15           -1.4689     0.9520           1.371712    0.0523   65.394330  
16            1.0985     2.2279           4.163969    0.0493   42.383126  
17           -0.0000     1.1966           1.917886    0.0391   26.983289  
18           -1.0351     0.9629           1.333980    0.0386   34.089623  
19           -1.1994     1.8333           1.499162    0.0350   80.082498  
20            0.8825    -0.0878          -2.294983    0.0232   38.878425  
21           -1.0792     1.8181           2.832215    0.0145   84.385727  
22           -0.9478     2.0551           2.879143    0.0105   81.978758  
23           -0.0000     0.4643           0.624507    0.0089    3.456579  
24           -0.0000     0.3304           0.429886    0.0084    1.610113  
25            0.0000    

<Figure size 1440x1080 with 0 Axes>

In [11]:
alphas = []
l1_ratios = []
i=0
results = results_nan.copy()

plt.figure(figsize=(20, 15))
plt.subplots_adjust(hspace=0.5)

for c in tracts_health.columns:
    if c.endswith('CrudePrev'):
        name = name_mapping[c[:-10].lower()]
        
        x = tracts_health[desert_measures].to_numpy(copy=True)
        scaler = StandardScaler()
        xscale = scaler.fit_transform(x)
        y = tracts_health[c].to_numpy(copy=True)
        xscale = xscale[~np.isnan(y)]
        y = y[~np.isnan(y)]
        #create function to center data
        center_function = lambda x: x - x.mean()

        #apply function to original NumPy array
        y = center_function(y)
        
        #l1 ratio is from suggested values in ElasticNetCV documentation
        lasso_cv = LassoCV(cv = 5, normalize=True).fit(xscale,y)
        
        alpha = lasso_cv.alpha_
        alphas.append(alpha)
        
        regr = Lasso(alpha=alpha, normalize=True)  # Could try others, or other parameters?
        regr.fit(xscale, y.reshape(-1, 1))
        
        predictions = regr.predict(xscale)
        mse_test = mean_squared_error(y, predictions)
        
        results.iat[i, 0] = name
        results.iat[i, 1] = regr.coef_[1]
        results.iat[i, 2] = regr.coef_[3]
        results.iat[i, 3] = regr.coef_[4]
        results.iat[i, 4] = regr.coef_[0]
        results.iat[i, 5] = regr.coef_[2]
        results.iat[i, 6] = regr.score(X_test, y_test)
        results.iat[i, 7] = mse_test

        i += 1

results_round = results.round({'Food': 4, 'Physical health':4, 'Public transport':4, 'Education':4,
             'House of worship':4, 'RSquared':4}).sort_values(by='RSquared', ascending=False).reset_index(drop=True)
results_round.to_csv(save_results_name)
results_round

Health condition    Food  Physical health  \
0       Poor mental health prevalence -0.0000          -2.0619   
1                  Smoking prevalence  0.2022          -3.1096   
2             Health insurance access  0.5506          -3.6064   
3               Poor sleep prevalence -0.1260          -3.7958   
4                   Asthma prevalence -0.0000          -1.2328   
5                          Mammograms -0.2452          -1.0111   
6                     COPD prevalence  0.4366          -1.5438   
7                Poor physical health  0.6465          -2.9319   
8                   Stroke prevalence  0.2339          -1.3322   
9              Chronic kidney disease  0.1630          -0.8695   
10  Coronary heart disease prevalence  0.3988          -1.0475   
11              Depression prevalence -0.0000          -0.0000   
12                Diabetes prevalence  0.7098          -3.8394   
13                   Routine checkups  0.2045          -1.9413   
14                  Cancer prevalence  0.2720           0.0000   
15               Arthritis prevalence  1.1602          -2.5313   
16                 Obesity prevalence  0.2342          -5.4614   
17          Binge drinking prevalence -0.4382           1.9064   
18        High cholesterol prevalence  1.2107          -0.1616   
19               No physical activity  0.7662          -5.7308   
20              Teeth loss prevalence  0.2386          -5.5935   
21     General poor health prevalence  1.0037          -5.8013   
22         Cervical cancer screenings  0.0000           0.5914   
23             Cholesterol screenings  0.3489           0.0000   
24   Medium blood pressure prevalence  1.6350          -2.6650   
25     High blood pressure prevalence  1.0432          -6.5649   
26            Colon cancer screenings -0.1417           1.7823   
27                Core women's health -0.0000           3.6651   
28                  Core men's health -0.0000           4.3844   
29                    Dental checkups -0.0000           7.3148   

    Public transport  Education  Houses of worship  RSquared         MSE  
0            -0.7075     0.8717           0.750173    0.0937   14.641736  
1            -1.0135     1.8386           1.611182    0.0549   28.092201  
2            -1.4433     2.1183           1.752038    0.0510   51.149097  
3            -1.0859     1.7656           2.782301    0.0468   28.960101  
4            -0.1878     0.5452           0.655569    0.0432    2.700116  
5             0.0000     0.0000           0.665315    0.0394    1.217356  
6            -0.1121     1.0321           0.391139    0.0277    5.176842  
7            -0.4417     1.8618           1.036275    0.0260   16.908452  
8            -0.0000     0.8004           0.404737    0.0241    2.680505  
9            -0.0000     0.5280           0.247820    0.0176    1.266482  
10            0.0000     0.7438           0.117097    0.0167    3.622020  
11           -0.0000    -0.0000          -0.295825    0.0032    1.798056  
12           -0.1070     2.3336           1.493682   -0.0257   21.587364  
13            0.0548     1.1860           1.883154   -0.0316   10.353380  
14            0.2411     0.1380           0.000000   -0.0465    2.529378  
15            0.0278     2.0471           1.114791   -0.0499   25.544874  
16           -0.7487     2.7300           3.324958   -0.0653   41.626473  
17            0.0000    -1.4064          -1.243584   -0.0869    9.556851  
18            0.0362     1.1360           0.027494   -0.1078   19.923118  
19           -1.2258     3.3994           2.705054   -0.1155   64.291297  
20           -1.2018     3.2097           1.809405   -0.1166   69.883400  
21           -1.0826     3.4947           2.338104   -0.1302   62.922527  
22            0.5402    -0.0000           0.195856   -0.1331   11.334616  
23            0.6018     0.3195           0.642948   -0.1379   10.025381  
24            0.3672     2.4188           1.837283   -0.1666   58.336576  
25           -0.0000    

<Figure size 1440x1080 with 0 Axes>